In [1]:
import sys
import os
#sys.path.append('D:\\software_git_repos\\cofe-python-analysis-tools\\utils_meinhold')
sys.path.append('../utils_meinhold')
#sys.path.append('D:\\software_git_repos\\cofe-python-analysis-tools\\utils_zonca')
sys.path.append('../utils_zonca')
#sys.path.append('D:\\software_git_repos\\cofe-python-analysis-tools\\utils_zonca\\pointing')
sys.path.append('../utils_zonca/pointing')
#sys.path.append('D:\\software_git_repos\\greenpol')
sys.path.append('../')
#sys.path.append('D:\\software_git_repos\\greenpol\\telescope_control\\')
sys.path.append('../telescope_control')
#sys.path.append('D:\\software_git_repos\\greenpol\\telescope_control\\VtoT\\')
sys.path.append('../VtoT')
print os.getcwd()
import convert
import realtime_gp as rt
import numpy as np
import datetime 
import h5py
import pandas as pd
#from pointingtools import compute_parallactic_angle, altaz2ha 
from planets import getlocation, getpointing
import warnings
from astropy.coordinates import AltAz, Angle, EarthLocation, ICRS, SkyCoord, frame_transform_graph
from astropy import units as u
import ephem
import matplotlib.pyplot as plt
import time
import planets
from datetime import timedelta, datetime
import numpy.ma as ma

import Tkinter,tkFileDialog

from Tkinter import *
import ttk

import pickle
import glob

import cPickle

from scipy.signal import savgol_filter

%pylab

C:\software_git_repos\polaris\polaris_software\notebook
Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


C:\Users\akkap\anaconda3\envs\py27\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['Widget', 'Text', 'Button', 'datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
%matplotlib auto

Using matplotlib backend: TkAgg


In [3]:
def get_pointing_files(filelist=None, days=1):

    if filelist==None:
        flist = []
        for d in range(days):
            root=Tkinter.Tk()
            files = list(tkFileDialog.askopenfilenames(\
            initialdir='D://software_git_repos/greenpol/telescope_control/data_aquisition/pointing_data/',parent=root,title='Choose a set of files'))
            flist.append(files)
            root.destroy()
        filelist = [y for x in flist for y in x]
    filelist.sort()
    
    return filelist

In [4]:
def read_some_data(filelist=None, days=1):

    if filelist==None:
        flist = []
        for d in range(days):
            root=Tkinter.Tk()
            files = list(tkFileDialog.askopenfilenames(\
            initialdir='D://software_git_repos/greenpol/telescope_control/data_aquisition/demod_data/',parent=root,title='Choose a set of files'))
            flist.append(files)
            root.destroy()
        filelist = [y for x in flist for y in x]
    filelist.sort()
    
    dlist=[]
    for f in filelist:
        hf=h5py.File(f)
        dlist.append(hf['demod_data'])
    d=np.concatenate(dlist)
    hf.close() 

    datadict=d
        
    return datadict, filelist

In [5]:
def get_file_times(fld):
    startfile = fld[0][:43]+fld[0][49:-2]+'dat'
    endfile = fld[-1][:43]+fld[-1][49:-2]+'dat'
    
    #starttime = os.path.getctime(startfile)
    starttime= os.stat(startfile).st_mtime + 7*3600
    starttime = datetime.datetime.fromtimestamp(starttime)
    
    #endtime = os.path.getctime(endfile)
    endtime= os.stat(endfile).st_mtime + 7*3600
    endtime = datetime.datetime.fromtimestamp(endtime)
    
    return starttime, endtime

In [6]:
def plotnow_azrevsig(data, az, chan, var, jump=180., minmax=None,return_map=False):
    #flp=select_h5(fpath,yrmoday,st_hour,st_minute,ed_hour,ed_minute)
    #fld_demod, fld =select_dat(fpath,yrmoday,st_hour,st_minute,ed_hour,ed_minute)
    #i=0
    #while len(flp)<3:
    #	i+=1
    #	flp=select_h5(fpath,yrmoday,st_hour,int(st_minute)-i,ed_hour,int(ed_minute)+i)

    #pp=rt.get_h5_pointing(flp)
    ##dd=get_demodulated_data_from_list(fld,supply_index=supply_index)
    #dd=rt.get_all_demodulated_data(fld_demod, fld)
    #combined=rt.combine_cofe_h5_pointing(dd,pp)

    #synchronized data and az values
    az1 = az*1
    data1 = data*1
    steps = len(data1)

    #convert to temp for cryo sensors
    if chan == 12:
        data1 = convert.convert(data1, 'i')
    if chan == 13:
        data1 = convert.convert(data1, 'e')
    if chan == 14:
        data1 = convert.convert(data1, 'h')
    if chan == 15:
        data1 = convert.convert(data1, 'l')

    #resolution
    dx = 1.0
    dy = 1.0

    #set up empty lists to append each revolution to
    data = []
    az = []
    iaz = [0]
    rev = 0

    #determine indices in azimuth/data array which correspond to a new revolution of the telescope
    for i in range(steps):
        #round values to resolution for comparison later
        az1[i] = rt.round_fraction(az1[i], dx)
        if i > 0:
            if abs(az1[i] - az1[i-1]) >= jump:
                iaz.append(i)
                rev += 1

    #append each revolution array to a list	    
    for j in range(rev):
        az.append(az1[iaz[j]:iaz[j+1]])
        data.append(data1[iaz[j]:iaz[j+1]])

    #append the last revolution
    data.append(data1[iaz[-1]:])
    az.append(az1[iaz[-1]:])
    rev += 1

    print rev

    data = np.asarray(data)
    az = np.asarray(az)

    #create grid for plotting
    x, y = np.arange(0., 360.+dx, dx), np.arange(0., rev - 1 + dy, dy)
    AZ, REV = np.meshgrid(x, y)

    #set up empty array
    z = np.zeros(len(x)*len(y))
    sig = np.reshape(z, (len(y), len(x)))

    #small number for comparing floats
    epsilon = dx

    #fill signal array with data points
    for r in range(rev):
        for a in range(len(x)):
            #find indices where combined azimuth data fits on x grid
            idx = np.where(abs(az[r] - x[a]) < epsilon)[0]
            #if idx length is 0 this will create a mask on that point, in idx len > 1, avg data points in the same bin
            sig[r][a] = data[r][idx].mean()

    #mask invalid values, i.e. where there are no data points
    sig = ma.masked_invalid(sig)

    #change units on plot label
    if int(chan[2:]) < 12:
        unit = 'V'
    else:
        unit = 'K' 

    name = rt.chantoname(chan)
    
    if return_map:
        return sig, AZ, REV
    else:

        plt.pcolormesh(AZ, REV, sig, cmap='jet')
        plt.colorbar(label = 'Signal, %s' % unit)
        if minmax != None:
            plt.clim(minmax[0],minmax[1])
        else:  
            plt.clim(data1.min(),data1.max())    #plt.clim(-1.0,1.0)
        plt.axis([0., 360., 0., rev - 1])
        plt.ylabel('revolution #')
        plt.xlabel('azimuth (deg)')
        plt.title('%s %s' % (name, var))
        plt.grid()
        plt.show()

In [7]:
def plotnow_azelsig(data, az, el, chan, var, res, minmax=None, radec=False, supply_index=False):
    #flp=select_h5(fpath,yrmoday,st_hour,st_minute,ed_hour,ed_minute)
    #fld_demod, fld =select_dat(fpath,yrmoday,st_hour,st_minute,ed_hour,ed_minute)
    #i=0
    #while len(flp)<3:
    #	i+=1
    #	flp=select_h5(fpath,yrmoday,st_hour,int(st_minute)-i,ed_hour,int(ed_minute)+i)

    #pp=get_h5_pointing(flp)
    ##dd=get_demodulated_data_from_list(fld,supply_index=supply_index)
    #dd=get_all_demodulated_data(fld_demod, fld)	
    #combined=combine_cofe_h5_pointing(dd,pp)

    #synchronized data az and el values
    az1, el1 = az, el
    data = data

    #convert to temp for cryo sensors
    if chan == 12:
        data = convert.convert(data, 'i')
    if chan == 13:
        data = convert.convert(data, 'e')
    if chan == 14:
        data = convert.convert(data, 'h')
    if chan == 15:
        data = convert.convert(data, 'l')

    steps = len(data)

    #set az/el resolution
    dx = res
    dy = res

    #set up bins/grid
    if radec:
        x, y = np.arange(0., 360.+dx, dx), np.arange(-90., 90. + dy, dy)
    else:
        x, y = np.arange(0., 360.+dx, dx), np.arange(0., 90. + dy, dy)
    AZ, EL = np.meshgrid(x, y)

    #small number for comparing floats
    epsilon = 1e-6

    #set up matrix for signal 
    z1 = np.zeros(len(x)*len(y))
    sig = np.reshape(z1, (len(y), len(x)))

    #set up matrix for keeping track of data points in single bin for averaging
    z2 = np.zeros(len(x)*len(y))
    count = np.reshape(z2, (len(y), len(x)))

    for i in range(steps):

        #round az/el points for comparison with grid	    
        el1[i] = rt.round_fraction(el1[i], dy)
        az1[i] = rt.round_fraction(az1[i], dx)  

        #find where data points belong in grid
        iel = np.where(abs(y - el1[i]) < epsilon)[0][0]
        iaz = np.where(abs(x - az1[i]) < epsilon)[0][0]

        #add 1 each time data point lands in same bin
        count[iel][iaz] += 1

        #add total number of data values in bin
        sig[iel][iaz] = sig[iel][iaz] + data[i]  

    #mask 0 count values so they dont show up in color plot
    count = ma.masked_where(count == 0.0, count)

    #take average of all data points in single bin
    sig = sig/count
    
    try:
        #change units on plot label
        if int(chan[2:]) < 12:
            unit = 'V'
        else:
            unit = 'K' 
            
        name = rt.chantoname(chan)
    except:
        unit = 'C'
        name = chan

    

    plt.pcolormesh(AZ, EL, sig, cmap='jet')
    plt.colorbar(label = 'Signal, %s' % unit)
    if minmax != None:
        plt.clim(minmax[0],minmax[1])
    else:  
        plt.clim(data.min(),data.max())
    if radec == False:
        plt.axis([AZ.min(), AZ.max(), EL.min(), EL.max()])
        plt.ylabel('elevation (deg)')
        plt.xlabel('azimuth (deg)')
    else:
        plt.axis([0., 360., -90, 90.])
        plt.xlabel('ra (deg)')
        plt.ylabel('dec (deg)')
    plt.title('%s %s data binned to azimuth and elevation' % (name, var))
    plt.grid()
    plt.show()

In [8]:
def fix_pointing(data, az):

    az1 = az*1
    data1 = data*1
    steps = len(data1)

    #set up empty lists to append each revolution to
    data = []
    az = []
    iaz = [0]
    rev = 0

    #determine indices in azimuth/data array which correspond to a new revolution of the telescope
    for i in range(steps):
        if i > 0:
            if abs(az1[i] - az1[i-1]) >= 180.:
                iaz.append(i)
                rev += 1

    #append each revolution array to a list	    
    for j in range(rev):
        az.append(az1[iaz[j]:iaz[j+1]])
        data.append(data1[iaz[j]:iaz[j+1]])

    #append the last revolution
    data.append(data1[iaz[-1]:])
    az.append(az1[iaz[-1]:])
    rev += 1

    print rev
    
    data, az = np.asarray(data), np.asarray(az)
    
    return data, az

In [9]:
def fix_time(ut, maxjump):
    ijump = np.where(np.diff(ut/3600.)>maxjump)[0]
    ut_corrected = ut*1
    for i in ijump:
        diff = ut[i+1]-ut[i]
        ut_corrected[i+1:] = ut_corrected[i+1:]-diff
    
    dt_corrected = []
    for u in ut_corrected:
         dt_corrected.append(datetime.datetime.fromtimestamp(u))
        
    return dt_corrected, ut_corrected

In [42]:
ddict={}

In [44]:
dd, fld = read_some_data(days=1)

C:\Users\akkap\anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:15: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  from ipykernel import kernelapp as app


In [45]:
flp = get_pointing_files(days=1)

In [46]:
pp = rt.get_h5_pointing(flp)

In [47]:
combined = rt.combine_cofe_h5_pointing(dd, pp)

In [48]:
#lvl1f=h5py.File('D:/software_git_repos/greenpol/telescope_control/data_aquisition/level1/20180510/ch1-10_34_06-15_56_00.h5')['data']

In [49]:
starttime, endtime = get_file_times(fld)
dt , ut= rt.convert_gpstime(starttime, combined['gpstime']/1000, format='datetime')

In [50]:
dt, ut = fix_time(ut, maxjump=2)

In [51]:
plt.figure()
plt.plot((ut-ut[0])/3600.)

In [52]:
az, el = combined['az'], combined['el']

In [53]:
combined.keys()

['el',
 'Phidget Temp',
 'Cooler',
 'flag',
 'az',
 'gpstime',
 'sci_data',
 'Calibrator',
 'Amplifier',
 'x tilt',
 'y tilt',
 'Backend TSS']

In [54]:
chan = 'H3HiAC'
var = 'T'
data = combined['sci_data'][rt.nametochan(chan)][var]

In [55]:
sig, AZ, REV = plotnow_azrevsig(data, az, rt.nametochan(chan), var, jump=40., minmax = None, return_map=True)

440


C:\Users\akkap\anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:80: RuntimeWarning: Mean of empty slice.


In [56]:
plt.figure()
vmin = -0.2
vmax = 0#data.max()
plt.imshow(sig, vmin = vmin, vmax = vmax, cmap='jet', origin='lower', aspect='auto')
plt.xlabel('azimuth (deg)')
plt.ylabel('revolution #')
plt.title('%s %s data ' % (chan, var))
plt.grid()
plt.colorbar(label = 'Signal, V')

In [93]:
figure()
plotnow_azrevsig(data, az, rt.nametochan(chan), var, jump=180., minmax = [-0.2, 0.0])

426


C:\Users\akkap\anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:80: RuntimeWarning: Mean of empty slice.


In [24]:
figure()
plotnow_azelsig(data, az, el, rt.nametochan(chan), var, 0.1, radec=False, minmax = [data.min(), data.max()])

C:\Users\akkap\anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:72: RuntimeWarning: invalid value encountered in divide


In [94]:
utabs = ut - ut[0]
utmin = 66.*60
utmax = 298.*60
azmin = 164.
azmax = 258.

idata = data[(az>azmin) & (az<azmax) & (utabs<utmax) & (utabs>utmin)]
iut = ut[(az>azmin) & (az<azmax) & (utabs<utmax) & (utabs>utmin)]
#idt = dt[(az>azmin) & (az<azmax) & (utabs<utmax) & (utabs>utmin)]
iaz = az[(az>azmin) & (az<azmax) & (utabs<utmax) & (utabs>utmin)]
iel = el[(az>azmin) & (az<azmax) & (utabs<utmax) & (utabs>utmin)]

In [95]:
lim=0.2
sig, AZ, REV = plotnow_azrevsig(idata[abs(idata)<lim], iaz[abs(idata)<lim], rt.nametochan(chan), var, jump=40., minmax = None, return_map=True)
#sig, AZ, REV = plotnow_azrevsig(data, az, rt.nametochan(chan), var, jump=40., minmax = None, return_map=True)

189


C:\Users\akkap\anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:80: RuntimeWarning: Mean of empty slice.


In [96]:
print len(REV)
slaz2 = []
slrev2 = []
slsig2 = []
for r in range(len(REV)):
    slaz2.append(AZ[0][sig[r] == sig[r].min()])
    slrev2.append(r+1)
    slsig2.append(sig[r].min())
slaz2 = np.asarray(slaz2).flatten()
slrev2 = np.asarray(slrev2).flatten()
slsig2 = np.asarray(slsig2).flatten()

189


In [97]:
idiff2 = np.where(abs(np.diff(slaz2))<=4.5)[0]
slaz2 = slaz2[idiff2][2:]
slrev2 = slrev2[idiff2][2:]
slsig2 = slsig2[idiff2][2:] 
slaz2 = savgol_filter(slaz2, 51, 3)

In [98]:
plt.figure()
plt.plot(slaz2, slrev2)

In [100]:
plt.figure()
vmin = -0.2
vmax = 0#data.max()
#plt.subplot(1,2,1)
plt.plot(slaz2, slrev2, 'k-', linewidth=3)
plt.imshow(sig, vmin = vmin, vmax = vmax, cmap='jet', origin='lower', aspect='auto')
plt.xlabel('azimuth (deg)')
plt.ylabel('revolution #')
plt.title('%s %s data ' % (chan, var))
plt.grid()
plt.colorbar(label = 'Signal, V')

In [105]:
azsun_interp = np.interp(slrev2, times*60, azsun)
elsun_interp = np.interp(slrev2, times*60, elsun)

azmoon_interp = np.interp(slrev2, times*60, azmoon)
elmoon_interp = np.interp(slrev2, times*60, elmoon)

In [107]:
suncoord = SkyCoord(az = azsun_interp, alt = elsun_interp, frame = 'altaz', unit='deg')
mooncoord = SkyCoord(az = azmoon_interp, alt = elmoon_interp, frame = 'altaz', unit='deg')

slcoord = SkyCoord(az = slaz2, alt = 70., frame = 'altaz', unit='deg')

angdiff_sun = slcoord.separation(suncoord).deg
angdiff_moon = slcoord.separation(mooncoord).deg
#suncoord = SkyCoord(azsun_interp, elsun_interp, frame='altaz')

In [108]:
sigmean = slsig2.mean()*-1.47
Tsun = 6000
ratio = sigmean/Tsun
db = 10*np.log10(ratio)
print db

-43.503672170271834


In [113]:
plt.figure()
plt.plot(angdiff_sun, slrev2, label='day2')
#plt.plot(angdiff2, slrev2, label='day 1')
plt.ylabel('rev #')
plt.xlabel('angular seperation (deg)')
plt.title('sun-sidelobe angular separation, signal brightness = %.2f dB' % db)
plt.legend()
plt.grid()

In [577]:
plt.figure()
plt.plot(np.full(len(slrev), 70.), slrev, label='scan elevation')
plt.plot(elsun_interp2, slrev2, label='sun elevation')
plt.xlabel('el (deg)')
plt.ylabel('rev #')
#plt.title('sun elevation during const el=70 deg scan')
plt.grid()
plt.legend()

In [345]:
plt.figure()
plt.plot(slaz, slrev, label='sidelobe az')

plt.plot(azsun_interp, slrev, label='sun az')
plt.xlabel('az (deg)')
plt.ylabel('rev #')
plt.title('Constant elevation scan at el=70 deg')
plt.legend()
plt.grid()

In [373]:
plt.figure()
plt.plot(slaz, slrev, 'k-', linewidth=3)

In [483]:
plt.figure()
vmin = -0.2
vmax = 0#data.max()
#plt.subplot(1,2,1)
#plt.plot(slaz2, slrev2, 'k-', linewidth=3)
plt.imshow(sig, vmin = vmin, vmax = vmax, cmap='jet', origin='lower', aspect='auto')
plt.xlabel('azimuth (deg)')
plt.ylabel('revolution #')
plt.title('%s %s data ' % (chan, var))
plt.grid()
plt.colorbar(label = 'Signal, V')

In [102]:
hours = len(REV)/60.
sph = 60
samples = hours*sph
times = np.linspace(0, hours, samples)
starttime = datetime.datetime.fromtimestamp(iut[0])#datetime.datetime(2020, 10, 7, 5, 5, 0)#datetime.datetime.utcnow()

azsun = []
elsun = []
azmoon = []
elmoon = []
#tsun = []

target = 'Sun'
target2 = 'Moon'

for t in times:
    deltaT = starttime+timedelta(0, t*3600.)
    azs, els = getpointing('Sedgwick', target, deltaT)
    azsun.append(azs)
    elsun.append(els)
    #tsun.append(time.mktime(deltaT.timetuple()))
    azm, elm = getpointing('Sedgwick', target2, deltaT)
    azmoon.append(azm)
    elmoon.append(elm)
    
azsun = np.asarray(azsun)
elsun = np.asarray(elsun)
#tsun = np.asarray(tsun)
#azsun = np.interp(np.arange(len(REV)), )
azmoon = np.asarray(azmoon)
elmoon = np.asarray(elmoon)

C:\Users\akkap\anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.
  after removing the cwd from sys.path.
        Use `lon` instead. [astropy.utils.decorators]
        Use `lat` instead. [astropy.utils.decorators]


In [86]:
 
location = getlocation('Sedgwick')

t1 = time.time()
#create ra dec sky object
azel = SkyCoord(az = az, alt = el, obstime = dt, location = location, frame = 'altaz', unit='deg')
t2 = time.time()
print t2-t1
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    #convert from ra dec to az/el for pointing
    radec = azel.icrs
print time.time()-t2
ra = radec.ra.deg
dec = radec.dec.deg

0.642000198364
33.0609998703


In [11]:
#azsuntrue, altsuntrue = getpointing('UCSB', 'Sun', datetime.datetime.fromtimestamp(time.time()))
azsuntrue, altsuntrue = getpointing('UCSB', 'Sun', datetime.datetime.utcnow())

print altsuntrue, azsuntrue

28.9440277778 230.28475


In [36]:
cra = 15*(5 + (34./60.) + (31.94/3600.))
cdec = 22 + (52.2/3600.)

caz, cel = planets.radec_to_azel(cra, cdec, 'UCSB', datetime.datetime.utcnow())

In [31]:
print cdec, cra
print cel, caz

22.0145 83.6330833333
52.1944601993 98.1316757854


In [41]:
hours = len(REV)/60.
sph = 60
samples = hours*sph
times = np.linspace(0, hours, samples)
starttime = dt[0]#datetime.datetime(2020, 10, 7, 5, 5, 0)#datetime.datetime.utcnow()

azsun = []
elsun = []
azmoon = []
elmoon = []

target = 'Venus'

for t in times:
    azs, els = getpointing('Sedgwick', target, starttime+timedelta(0, t*3600.))
    azsun.append(azs)
    elsun.append(els)
    #azm, elm = getpointing('UCSB', [cra, cdec, 'RADEC'], datetime.datetime.utcnow()+timedelta(0, t*3600.))
    #azmoon.append(azm)
    #elmoon.append(elm)
    
azsun = np.asarray(azsun)
elsun = np.asarray(elsun)
#azmoon = np.asarray(azmoon)
#elmoon = np.asarray(elmoon)

C:\Users\akkap\anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.
  after removing the cwd from sys.path.


In [42]:
figure()
plot(elsun, times, 'r-',label=target)
#plot(times, elmoon, 'b-',label='moon')
title('%s el' % target)
xlabel('time (hours)')
ylabel('el (deg)')
legend()

figure()
plot(azsun, times, 'r-',label=target)
#plot(times, azmoon, 'b-',label='moon')
title('%s az' % target)
xlabel('time (hours)')
ylabel('az (deg)')
legend()
'''
figure()
dist = np.sqrt((((azsun - azmoon)**2) + ((elsun - elmoon)**2))) 
plot(times, dist, 'r-',label='distance')
title('sun/moon distance apart')
xlabel('time (hours)')
ylabel('distance (deg)')
legend()
'''

"\nfigure()\ndist = np.sqrt((((azsun - azmoon)**2) + ((elsun - elmoon)**2))) \nplot(times, dist, 'r-',label='distance')\ntitle('sun/moon distance apart')\nxlabel('time (hours)')\nylabel('distance (deg)')\nlegend()\n"

In [154]:
ra = (5. + 34./60. + 31.94/3600.)*360./24.
dec = 22 + 52.2/3600.

azt, elt = planets.radec_to_azel(ra, dec, 'Greenland', datetime.datetime.utcnow())


print ra, dec
print azt, elt

83.6330833333 22.0145
268.305836314 23.6730372224


In [28]:
ra = (5. + 34./60. + 31.94/3600.)*360./24.
dec = 22 + 52.2/3600.

hours = len(REV)/60.
sph = 60
samples = hours*sph
times = np.linspace(0., hours, samples)

aztau = []
eltau = []

for t in times:
    #azt, elt = getpointing('Greenland', [ra, dec, 'RADEC'], datetime.datetime.utcnow()+timedelta(0, t*3600.))
    azt, elt = planets.radec_to_azel(ra, dec, 'Sedgwick', dt[0]+timedelta(0, t*3600.))
    aztau.append(azt)
    eltau.append(elt)

C:\Users\akkap\anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.
  import sys


In [31]:
figure()
plot(aztau,times, 'g-')
title('tau a az')
ylabel('time (hours)')
xlabel('az (deg)')

figure()
plot(eltau, times, 'g-')
title('tau a el')
ylabel('time (hours)')
xlabel('el (deg)')

'''
figure()
dist = np.sqrt((((azsun - aztau)**2) + ((elsun - eltau)**2))) 
plot(times, dist, 'r-',label=' tau/sun distance')
xlabel('time (hours)')
ylabel('distance (deg)')

dist2 = np.sqrt((((aztau - azmoon)**2) + ((eltau - elmoon)**2))) 
plot(times, dist2, 'g-',label='tau/moon distance')
xlabel('time (hours)')
ylabel('distance (deg)')
legend()
'''

"\nfigure()\ndist = np.sqrt((((azsun - aztau)**2) + ((elsun - eltau)**2))) \nplot(times, dist, 'r-',label=' tau/sun distance')\nxlabel('time (hours)')\nylabel('distance (deg)')\n\ndist2 = np.sqrt((((aztau - azmoon)**2) + ((eltau - elmoon)**2))) \nplot(times, dist2, 'g-',label='tau/moon distance')\nxlabel('time (hours)')\nylabel('distance (deg)')\nlegend()\n"